In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf

In [2]:
# resetting the seeds for reproducibility
def reset_random_seeds():
    n = 1
    os.environ['PYTHONHASHSEED'] = str(n)
    tf.random.set_seed(n)
    np.random.seed(n)
    random.seed(n)

reset_random_seeds()

# import data
df = pd.read_csv('Boruta_onchain_data.csv')
df1 = pd.read_csv('all_data.csv')

df = df[df['timestamp'] >= '2013-03-11'].reset_index(drop=True)

#onchain_data_new
#TA_data
#all_data
#Boruta_data
#Boruta_TA_data
#Boruta_onchain_data

# separate the inputs and target
X = df.drop('timestamp', axis=1)

# create binary classification for price movement. this assigns 1 to y if price will move upward next day.
price = pd.DataFrame()
price['today'] = df1['price-ohlc-usd-c']
price['next day'] = price['today'].shift(-1)
y = price['next day']

# Drop the last row where 'next day' would be NaN after shifting
X = X[:-1]
y = y.dropna()

# separate training data from testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [3]:
# scale the input data
scaler = StandardScaler()

# Reshape X_train and X_test if they are 1D
if X_train.ndim == 1:
    X_train = X_train.to_numpy().reshape(-1, 1)
if X_test.ndim == 1:
    X_test = X_test.to_numpy().reshape(-1, 1)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# reshape the input data for CNN-LSTM (samples, timesteps, features)
def create_sequences(data, timesteps):
    X = []
    for i in range(len(data) - timesteps + 1):
        X.append(data[i:i + timesteps])
    return np.array(X)

timesteps = 5
X_train_reshaped = create_sequences(X_train_scaled, timesteps)
X_test_reshaped = create_sequences(X_test_scaled, timesteps)
y_train = y_train[timesteps - 1:]
y_test = y_test[timesteps - 1:]

In [4]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=256, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
model.add(TimeDistributed(Dense(1, activation='linear')))
model.add(Dropout(0.2))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=64))
model.add(Dropout(0.2))
model.add(Dense(64, activation='linear'))
model.add(Dense(1, activation='linear'))

# compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_absolute_error', 'mean_squared_error'])

C:\Users\oluwa\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
model.save("lstm_r_model.h5")

In [5]:
# train with the training dataset
early_stopping = EarlyStopping(monitor='mean_absolute_error', patience=50)
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=50, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100
54/54 [==============================] - 6s 35ms/step - loss: 27312638.0000 - mean_absolute_error: 3446.6111 - mean_squared_error: 27312638.0000 - val_loss: 1316821632.0000 - val_mean_absolute_error: 31361.2461 - val_mean_squared_error: 1316821632.0000
Epoch 2/100
54/54 [==============================] - 1s 16ms/step - loss: 26699382.0000 - mean_absolute_error: 3359.4253 - mean_squared_error: 26699382.0000 - val_loss: 1309872768.0000 - val_mean_absolute_error: 31250.2715 - val_mean_squared_error: 1309872768.0000
Epoch 3/100
54/54 [==============================] - 1s 17ms/step - loss: 25822262.0000 - mean_absolute_error: 3248.9490 - mean_squared_error: 25822262.0000 - val_loss: 1299842432.0000 - val_mean_absolute_error: 31089.3789 - val_mean_squared_error: 1299842432.0000
Epoch 4/100
54/54 [==============================] - 1s 16ms/step - loss: 24654018.0000 - mean_absolute_error: 3145.1670 - mean_squared_error: 24654018.0000 - val_loss: 1286897664.0000 - val_mean_absolute_

In [6]:
# Make predictions on the test set
y_pred = model.predict(X_test_reshaped)

# evaluate the prediction performance
print("Root Mean Squared Error:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("R^2 Score:", r2_score(y_test, y_pred))

24/24 [==============================] - 1s 7ms/step
Root Mean Squared Error: 21430.050190527883
Mean Absolute Error: 17203.97245068391
R^2 Score: -1.813263554660701


In [7]:
time = df['timestamp']
time_train, time_test = train_test_split(time, test_size=0.2, shuffle=False)
time_test = time_test[timesteps:]

# Flatten y_pred to be a 1-dimensional array
y_pred_flat = y_pred.flatten()

# Create a DataFrame with columns time_test, y_test, and y_pred
pred_res = pd.DataFrame({'date': time_test, 'actual': y_test.values, 'prediction': y_pred_flat, 'value': price['today'][-748:]})

pred_res.to_csv('pred2/lstm_uni_data.csv', index=False)